# Project - Apache Spark & Elastichsearch

##### Students:
* Lilia IZRI      (DS)
* Yacine MOKHTARI (DS)
* Alexandre COMBEAU (DS)

##### Report
[PENSER A METTRE UN LIEN ICI]


In [3]:
# import necessary packages
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext


# For ML
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.clustering import StreamingKMeans

# From our util.py file
from utils import sentiment, tweetToJSON
import textblob

In [6]:
import elasticsearch
elasticsearch.__version__

(8, 2, 0)

## I. Process & Analyze input data (tweets)
### 1. Create our Dstream that receives data

In [ ]:
# Initiate the SparkContext and StreamingContext with 10 second batch interval
sc = SparkContext()
ssc = StreamingContext(sc, 10)
spark = SparkSession(sc)
ssc.checkpoint("file:///tmp/spark")    # Checkpoint for backups (useful for operations by window)

In [ ]:
# initiate streaming text from a TCP (socket) source (Our tweets received)
socket_stream = ssc.socketTextStream("127.0.0.1", 5567)

### 2. Process data and tag with sentiment 

Here, we just took into account the polarity and choosed to ignore the subjectivity !  ;)

In [ ]:
# We split the fields of the tweet received and we add tag the data with the sentiment of the tweet
#   so the rdd below 'tweets' will be of the form (user, text, date, latitude, locations, hashtags, sentiment, tweet_id)
def mapSplit(tweet):
    """
    A function that takes a tweet  (the one we sent from the other iPython file),
    splits it into its different fields and adds the sentiment field {-1, 0, 1}
    """
    return (tweet[1],    tweet[2],  tweet[3],    tweet[4],    tweet[5],    tweet[6],   "sentiment: "+ str(sentiment(tweet[2][6:])), tweet[7])
             #user        #text       #date      #latitude   #longitude    #Hashtags    #sentiment(= {-1,0,1})   #Id

In [ ]:
#### 1. Process the received tweets ( we will catch them the same way we sent them into the socket  :  " ###field### field_name: ... ### ...."
tweets_split = socket_stream.map(lambda tweet: tweet.split(' ###:field:### '))
tweets = tweets_split.map(mapSplit)

#### 2. Tweets contains RDDs representented as tuples
# tweets.pprint() # uncomment this line to see the tweets in the tuple format

#### 3. json_list_per_stream is a list of tweets tuples converted  as a string following the JSON/Dict format
json_list_per_stream = tweets.map(tweetToJSON)
# json_list_per_stream.pprint() # uncomment this line to see the tweets in the JSON format

Form of RDD in Tweets :
``('user: userX', 'tweet: @userY blablablbabla', 'date: Thu May 05 00:16:06 +0000 2022', 'lat: 44.933143', 'lon: 7.540121', 'hashtags: #SaveTheWorld', 0, "1237288393929")``

### 3. ML : Cluster tweets according to sentiments and their location

In [ ]:
# We create a training set and test set 
training_data =  tweets.map(lambda tweet: Vectors.dense([float(tweet[6]), float(tweet[3][5:]), float(tweet[4][5:])]))
testing_data  =  tweets.map(lambda tweet: LabeledPoint(float(tweet[6]), Vectors.dense([float(tweet[6]), float(tweet[3][5:]), float(tweet[4][5:])])))


# We create a model with random clusters and specify the number of clusters to find
k = 3
dimension = 3
weights = 0.0
seed = 21

# init
model = StreamingKMeans(k=k, decayFactor=0.5).setRandomCenters(dimension, weights, seed)

# Train the model
model.trainOn(training_data)  

# Predict
result = model.predictOnValues(testing_data.map(lambda lp: (lp.label, lp.features)))
# result.pprint()

In [ ]:
# We keep the predictions of each tweet (the index of the cluster), and we create (indexCluster, 1) pairs
predictions   = result.map(lambda x: (x[1], 1))

# We reduce by key and window to get the number of elements assigned to each cluster
size_clusters = predictions.reduceByKeyAndWindow(lambda x, y: x + y, lambda x, y: x - y, 30, 10)
size_clusters.pprint()

In [ ]:
# tweets.saveAsTextFiles("tmp/")

In [ ]:
# start streaming and wait couple of minutes to get enought tweets
ssc.start()

In [ ]:
print("helloooo", json_list_per_stream)

In [ ]:
print("Clusters coordinates: " + str(model.latestModel().clusterCenters))

In [ ]:
print("Clusters coordinates: " + str(model.latestModel().clusterCenters))

In [ ]:
## La bizarrement ça marche tout seul... mais quand c'est un stream ça plante


# a = "id: 234774849593"

# es.index(index="hello",
#                 id=int(a[4:]),
#                 document={"user": "lili",
#                           "text": "22"})